# Palia — Monetization & Retention Charts
This notebook runs root-level cell scripts to generate charts. Ensure Cell 1 (SQL) has produced `df_daily`, then run all cells.

In [ ]:
%run ../..//cell_style.py

In [ ]:
%run ../..//cell_2_summary.py

In [ ]:
%run ../..//cell_5_mac_arpmac_matrix.py

In [ ]:
%run ../..//cell_6_retention_periods.py

In [ ]:
%run ../..//cell_6_customer_retention_cohorts.py

In [ ]:
%run ../..//cell_7_conversion_charts.py

In [ ]:
%run ../..//cell_8_mtx_mix_charts.py

In [ ]:
%run ../..//cell_9_ltv_charts.py

In [ ]:
%run ../..//cell_10_activity_mix_customers_share.py

In [ ]:
%run ../..//cell_11_vc_price_realization.py

In [ ]:
%run ../..//cell_12_dormancy_rate.py